In [ ]:
import boto3
bucket_name = 'dataminds-warehouse'
s3_file_key = 'ramen-ratings.csv'            # e.g. 'folder/myfile.txt'
local_file_path = 'ramen-ratings.csv'        # Local destination

# Create an S3 client (remove `bucket_name` here — not a valid argument for boto3.client)
s3 = boto3.client(
    's3',
    region_name='us-east-1'
)

# Download the file
try:
    s3.download_file(bucket_name, s3_file_key, local_file_path)
    print(f"✅ File downloaded successfully from s3://{bucket_name}/{s3_file_key} to {local_file_path}")
except Exception as e:
    print("❌ Error downloading file:", e)


'import boto3\nbucket_name = \'dataminds-warehouse\'\ns3_file_key = \'ramen-ratings.csv\'            # e.g. \'folder/myfile.txt\'\nlocal_file_path = \'ramen-ratings.csv\'        # Local destination\n\n# Create an S3 client (remove `bucket_name` here — not a valid argument for boto3.client)\ns3 = boto3.client(\n    \'s3\',\n    region_name=\'us-east-1\'\n)\n\n# Download the file\ntry:\n    s3.download_file(bucket_name, s3_file_key, local_file_path)\n    print(f"✅ File downloaded successfully from s3://{bucket_name}/{s3_file_key} to {local_file_path}")\nexcept Exception as e:\n    print("❌ Error downloading file:", e)\n'

In [2]:
import pandas as pd
df = pd.read_csv('ramen-ratings.csv')
df.set_index('Review #', inplace=True)
df.tail()

,Brand,Variety,Style,Country,Stars,Top Ten
Review #,,,,,,
5,Vifon,"Hu Tiu Nam Vang [""Phnom Penh"" style] Asian Sty...",Bowl,Vietnam,3.5,NaN
4,Wai Wai,Oriental Style Instant Noodles,Pack,Thailand,1,NaN
3,Wai Wai,Tom Yum Shrimp,Pack,Thailand,2,NaN
2,Wai Wai,Tom Yum Chili Flavor,Pack,Thailand,2,NaN
1,Westbrae,Miso Ramen,Pack,USA,0.5,NaN


In [3]:
#df['Top Ten'].isna().sum()-2539 NaN values
df.drop('Top Ten',axis=1,inplace=True)

In [4]:
df.columns=df.columns.str.lower()

In [5]:
df.isna().sum()

brand      0
variety    0
style      2
country    0
stars      0
dtype: int64

In [6]:
df.dtypes

brand      object
variety    object
style      object
country    object
stars      object
dtype: object

In [7]:
df[df['stars']=='Unrated']

,brand,variety,style,country,stars
Review #,,,,,
2548,Ottogi,Plain Instant Noodle No Soup Included,Pack,South Korea,Unrated
2458,Samyang Foods,Sari Ramen,Pack,South Korea,Unrated
1587,Mi E-Zee,Plain Noodles,Pack,Malaysia,Unrated


In [8]:
import numpy as np
df['stars'] = df['stars'].replace("Unrated", np.nan)

In [9]:
df['stars'] = df['stars'].fillna(df['stars'].mode()[0])

In [10]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

# Fill missing variety texts (if any)
df['variety'] = df['variety'].fillna('missing')

# Encode all sentences at once (returns a 2D numpy array)
embeddings = model.encode(df['variety'].tolist())

# Convert embeddings array into DataFrame with multiple columns
embedding_df = pd.DataFrame(embeddings,index=[i for i in range(1,len(embeddings)+1)], columns = [f'emb_{i}' for i in range(1, embeddings.shape[1] + 1)])

2025-08-05 17:48:41.797549: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
embedding_df

,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,emb_10,...,emb_375,emb_376,emb_377,emb_378,emb_379,emb_380,emb_381,emb_382,emb_383,emb_384
1,0.008211,0.040726,-0.012296,-0.030254,-0.087250,-0.000035,0.117528,-0.032520,0.056304,-0.023947,...,-0.009106,-0.038584,0.068323,0.025785,-0.020410,-0.013544,0.041941,-0.001203,-0.006340,-0.047362
2,-0.145963,-0.049212,-0.021097,0.052561,-0.039781,0.014862,0.056952,-0.033205,0.073204,-0.094664,...,-0.048551,-0.067750,0.043449,-0.010555,0.031077,-0.070400,0.021824,0.062235,-0.045363,0.100652
3,-0.121339,-0.032021,-0.038093,0.049455,0.020214,0.037210,0.091100,0.021062,0.105264,-0.082693,...,-0.003368,0.059473,0.032751,0.010563,-0.029418,-0.015093,-0.107990,0.049097,0.088203,0.043062
4,-0.055996,-0.031858,-0.009268,-0.045139,-0.043832,-0.008629,0.132931,-0.015871,0.013650,-0.073199,...,0.068682,-0.035058,-0.052412,-0.032244,0.037478,-0.004841,-0.044936,-0.003457,0.023023,-0.009150
5,-0.017170,0.015357,-0.006900,0.026682,-0.026135,0.023832,0.103167,-0.015255,0.035849,0.022696,...,0.001207,-0.018672,-0.020922,-0.047290,0.001673,0.064595,0.106685,-0.008627,-0.025232,0.025313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2576,-0.079895,0.022555,0.012004,0.033568,-0.070995,-0.001656,0.087798,-0.051410,0.015902,-0.050165,...,-0.014823,0.053413,-0.000696,-0.023479,0.035456,-0.013986,0.031379,0.096622,-0.023745,0.060604
2577,-0.139653,-0.025586,-0.002901,0.099757,-0.070216,-0.014625,0.101016,-0.010362,0.056137,-0.121992,...,-0.047412,0.072332,0.019349,-0.029117,0.063175,0.001268,-0.013387,0.015482,-0.000821,0.066207
2578,-0.126256,0.032873,0.000950,-0.077178,-0.063134,-0.028888,0.070675,-0.038352,0.054752,-0.024734,...,-0.039208,-0.014450,0.026240,0.060703,0.000836,-0.004264,-0.006572,0.015426,0.054819,0.006474
2579,-0.127946,-0.023724,0.013316,-0.058578,-0.044750,0.036498,0.036504,-0.012537,0.048257,-0.099970,...,-0.011772,-0.012394,0.021764,0.025045,-0.006425,-0.045526,-0.024698,0.009744,0.055190,-0.048107


In [12]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
X = embedding_df.values
X_std = StandardScaler().fit_transform(X)
pca = PCA(n_components=100)
X_pca = pca.fit_transform(X_std)

In [13]:
df_reduced = pd.DataFrame(X_pca,index=[i for i in range(1,2580+1)])

In [14]:
df = pd.concat([df.drop(columns=['variety']), df_reduced], axis=1)
df.columns = df.columns.astype(str)

In [15]:
X=df.drop('stars',axis=1)
y=df['stars']
numeric_features=X.columns[X.dtypes=='float32']
categorical_features=['brand','style','country']

In [ ]:
!pip install sentence-transformers

In [ ]:
!pip install category_encoders

In [25]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler
from category_encoders import CatBoostEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import chi2
from sklearn.impute import SimpleImputer


#train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", RobustScaler()),
    ]
)
categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy='most_frequent')),
        ("encoder", CatBoostEncoder()),
        ("selector", SelectPercentile(chi2, percentile=50))  
    ]
)
preprocessor = ColumnTransformer(
    transformers=[
         ("num", numeric_transformer, numeric_features),
         ("cat", categorical_transformer, categorical_features),
    ]
)

clf = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("classifier", RandomForestClassifier(max_depth=10,min_samples_split=5,n_estimators=400))
    ]
)
clf.fit(X_train, y_train)
print("model score: %.3f" % clf.score(X_test, y_test))

model score: 0.186


In [26]:
import boto3

# Replace with your actual credentials and info
bucket_name = 'dataminds-homeworks'
s3_file_key = 'vasif-orujzada-fe2.ipynb' 
local_file_path = 'vasif-orujzada-fe2.ipynb'

# Create an S3 client
s3 = boto3.client(
    's3'
)

# Upload the file
try:
    s3.upload_file(local_file_path, bucket_name, s3_file_key)
    print(f"File uploaded successfully to s3://{bucket_name}/{s3_file_key}")
except Exception as e:
    print("Error uploading file:", e)


File uploaded successfully to s3://dataminds-homeworks/vasif-orujzada-fe2.ipynb
